 Import Required Libraries

In [6]:
import os
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import to_categorical


ModuleNotFoundError: No module named 'keras'

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import to_categorical

ModuleNotFoundError: No module named 'keras'

Matplotlib is building the font cache; this may take a moment.

Image Resizing and Preprocessing

In [3]:
img_rows, img_cols = 300, 300
img_channels = 1

path1 = "images"
path2 = "images_resized"
os.makedirs(path2, exist_ok=True)

listing = os.listdir(path1)
num_samples = len(listing)
print("Number of samples:", num_samples)

for file in listing:
    im = Image.open(os.path.join(path1, file))
    img = im.resize((img_rows, img_cols))
    gray = img.convert('L')
    gray.save(os.path.join(path2, file), "JPEG")


Number of samples: 4233


Create Image Matrix

In [4]:
imlist = os.listdir(path2)
im1 = array(Image.open(os.path.join(path2, imlist[0])))
immatrix = array([array(Image.open(os.path.join(path2, im2))).flatten() for im2 in imlist], 'f')


Labeling and Shuffling Data

In [5]:
label = np.ones((num_samples,), dtype=int)
label[0:1260] = 0
label[1260:2716] = 1
label[2716:] = 2

data, Label = shuffle(immatrix, label, random_state=2)
train_data = [data, Label]


Display a Sample Image

In [ ]:
img = immatrix[167].reshape(img_rows, img_cols)
plt.imshow(img, cmap='gray')
plt.show()

print("Data shape:", train_data[0].shape)
print("Label shape:", train_data[1].shape)


Train-Test Split and Normalization

In [ ]:
X, y = train_data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols).astype('float32') / 255

Y_train = to_categorical(y_train, 3)
Y_test = to_categorical(y_test, 3)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


Build the CNN Model

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='valid', input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])


Model Training

In [ ]:
batch_size = 32
nb_epoch = 20

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.2)


Visualizing Accuracy and Loss

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Train vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])
plt.grid(True)
plt.style.use('classic')
plt.show()

plt.figure(figsize=(7, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Train vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc=4)
plt.grid(True)
plt.style.use('classic')
plt.show()


 Evaluate Model

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test Score:', score[0])
print('Test Accuracy:', score[1])

print("Predicted Classes:\n", model.predict(X_test[1:5]).argmax(axis=1))
print("True Labels:\n", Y_test[1:5])


Classification Report & Confusion Matrix

In [ ]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
print("Classification Report:")
print(classification_report(np.argmax(Y_test, axis=1), y_pred, target_names=['Negative', 'Neutral', 'Positive']))
print("Confusion Matrix:")
print(confusion_matrix(np.argmax(Y_test, axis=1), y_pred))
